In [2]:
import pandas as pd
import yfinance as yf

In [3]:
# Baixa os dados da ação desejada
df_weekly = yf.download('^BVSP', interval='1wk')

[*********************100%***********************]  1 of 1 completed


In [4]:
def encontrar_maiores_altas(series):
    df = pd.DataFrame({'Close': series})
    df['Retorno'] = df['Close'].pct_change()
    
    periodos = []
    maiores_altas = []
    num_semanas_alta = []
    
    inicio_periodo = None
    maior_alta = 0
    semanas_alta = 0
    
    for index, row in df.iterrows():
        if row['Retorno'] > 0:
            if inicio_periodo is None:
                inicio_periodo = index
            maior_alta += row['Retorno']
            semanas_alta += 1
        else:
            if inicio_periodo is not None:
                periodos.append((inicio_periodo, index))
                maiores_altas.append(maior_alta)
                num_semanas_alta.append(semanas_alta)
                inicio_periodo = None
                maior_alta = 0
                semanas_alta = 0
    
    if inicio_periodo is not None:
        periodos.append((inicio_periodo, df.index[-1]))
        maiores_altas.append(maior_alta)
        num_semanas_alta.append(semanas_alta)
    
    df_maiores_altas = pd.DataFrame({'Inicio': [inicio for inicio, _ in periodos],
                                     'Fim': [fim for _, fim in periodos],
                                     'Maior Alta (%)': maiores_altas,
                                     'Num Semanas Alta': num_semanas_alta})
    
    df_maiores_altas = df_maiores_altas.sort_values(by='Num Semanas Alta', ascending=False)
    
    return df_maiores_altas

In [5]:
df_maiores_altas = encontrar_maiores_altas(df_weekly['Close'])
print(df_maiores_altas)

        Inicio        Fim  Maior Alta (%)  Num Semanas Alta
135 2003-09-29 2004-01-12        0.423178                15
3   1993-10-25 1994-01-24        1.531292                13
82  1999-10-18 2000-01-03        0.439406                11
160 2005-11-14 2006-01-30        0.218330                11
0   1993-05-03 1993-07-12        0.946974                10
..         ...        ...             ...               ...
195 2008-07-07 2008-07-14        0.013190                 1
196 2008-07-28 2008-08-04        0.007535                 1
197 2008-08-18 2008-08-25        0.029607                 1
11  1994-10-24 1994-10-31        0.080164                 1
194 2008-05-26 2008-06-02        0.015969                 1

[388 rows x 4 columns]


In [6]:
df_filtrado = df_maiores_altas.loc[df_maiores_altas['Inicio'].dt.year >= 2010]

df_filtrado.head(10)


,Inicio,Fim,Maior Alta (%),Num Semanas Alta
354,2020-11-02,2021-01-11,0.292242,10
301,2016-06-13,2016-08-22,0.181444,10
387,2023-04-24,2023-06-19,0.144885,9
315,2017-07-24,2017-09-18,0.159975,8
223,2010-08-30,2010-10-18,0.091811,7
331,2018-12-24,2019-02-04,0.134696,6
325,2018-06-25,2018-08-06,0.143976,6
319,2017-12-18,2018-01-29,0.166786,6
241,2012-01-02,2012-02-06,0.141642,5
328,2018-10-01,2018-11-05,0.109797,5
